In [ ]:
import nuclio 
import mlrun

In [ ]:
mlrun.get_version()

In [ ]:
import storey

In [ ]:
%nuclio env -c V3IO_ACCESS_KEY=${V3IO_ACCESS_KEY}
%nuclio env -c V3IO_USERNAME=${V3IO_USERNAME}
%nuclio env -c V3IO_API=${V3IO_API}

In [ ]:
%%nuclio cmd -c
pip install beautifulsoup4
pip install pandas
pip install v3io_frames

In [ ]:
%%nuclio config 
kind = "nuclio"
spec.build.baseImage = "mlrun/ml-models"

In [ ]:
%nuclio cmd -c pip install v3io

In [ ]:
# nuclio: start-code

In [ ]:
import mlrun.feature_store as fs
from mlrun.feature_store.steps import *
import mlrun
import pandas as pd
import numpy
import os
import math

In [ ]:
def rm_inf(data):
    print(data)
    data = [0 if item == math.inf or item == -math.inf else item for item in data ]
    #data = [0 for item in data if item == -math.inf]
    return data

In [ ]:
def handler(context, event):
    context.logger.info("Reading features from feature vector")
    stock_list = []
    # Reading the data from feature service
    service = fs.get_online_feature_service("stocks-vec")
    for key,value in context.sym_to_url.items():
        context.logger.info(key)
        data = service.get([{"symbol": key}])[0]
        data["symbol"] = key
        stock_list.append(data)
    
    data = [rm_inf(list(item.values())) for item in stock_list]
    
    columns = [{'text': key, 'type': 'object'} for key in stock_list[0].keys()]
    
    response = [{'columns': columns,
                'rows': data,
                'type': 'table'}]
    return response

In [ ]:
def init_context(context):
    # Setting our enviornment
    context.logger.info("Initalizing context & constructing featrure vector")
    setattr(context, 'PROJECT_NAME', os.getenv('PROJECT_NAME', 'stocks-' + os.getenv('V3IO_USERNAME')))
    mlrun.set_environment(project=context.PROJECT_NAME)
    
    sym_to_url = {'GOOGL': 'google-inc', 'MSFT': 'microsoft-corp', 'AMZN': 'amazon-com-inc',
                  'AAPL': 'apple-computer-inc', 'INTC' : 'intel-corp'}
    setattr(context, 'sym_to_url', sym_to_url)
    setattr(context, 'stocks_vec', "stocks-vec")
    
    # Declaring the features we would like to add to our vector
    features = ["stocks.prices_min_1h",
                "stocks.prices_max_1h",
                "stocks.volumes_min_1h",
                "stocks.volumes_max_1h",
                "stocks.price",
                "stocks.volume",
                "stocks.last_updated",
                "news.sentiments_min_1h",
                "news.sentiments_max_1h",
                "news.sentiment",
                "news.last_reaction"]
    
    # Making our vector
    vector = fs.FeatureVector(context.stocks_vec, features, description="stocks demo feature vector")
    vector.save()

In [ ]:
# nuclio: end-code

In [ ]:
# test local
init_context(context)
event = ""
df = handler(context,event)

In [ ]:
# test remote deployment
from mlrun import code_to_function
project_name = "stocks-" + os.getenv('V3IO_USERNAME')
fn = code_to_function('vector_reader',
                      handler='handler')

# Set parameters for current deployment
fn.set_envs({'PROJECT_NAME' : project_name,
             'STOCKS_STREAM':  os.getenv('V3IO_USERNAME') +' /stocks/stocks_stream'})
fn.spec.max_replicas = 2

In [ ]:
addr = fn.deploy(project=project_name)

In [ ]:
!curl {addr}